In [3]:
import gc
import h5py
import numpy as np
import pandas as pd

import h5py

import sys

from sklearn.model_selection import ParameterGrid


import os

sys.path.append("/home/nbbwang/bigdrive/AD_Project/")
from IntegratedGradients import *

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [4]:
SPECIFIC_FOLDER = "resid_01_GE_PmiRin"
SPECIFIC_FOLDER = "NEW_resid_then_01"
SPECIFIC_FOLDER = "origGE"


# SPECIFIC_FOLDER = "resid_01_GELabels_PmiRin"

datatype = "ACT_MSBBRNA_ROSMAP_PCASplit"

# path_to_split_data = "../MTL_data/%s/"%datatype
path_to_split_data = "../MTL_data/%s/ACT_MSBBRNA_ROSMAP_PCASplit/"%SPECIFIC_FOLDER 


num_components = 500


In [5]:
def kmeans_centroids_for_test(X_test, cluster_labels):
    n_clusters = len(np.unique(cluster_labels))
    n,d = X_test.shape
    
    new_centroids = np.zeros([n_clusters,n])
    for i in range(n_clusters):
        new_centroids[i] = np.mean(X_test.T[np.where(cluster_labels ==i)], axis=0).reshape([1,-1])

    return new_centroids

# Train / Test folds

In [23]:
mode = "TEST"
# mode = "TRAIN"


if mode =="TRAIN":
    path_to_new_files = "last_shared_layer_transformations_TRAIN/%s/unsupervised_methods/"%SPECIFIC_FOLDER
else:
    path_to_new_files = "last_shared_layer_transformations/%s/unsupervised_methods/"%SPECIFIC_FOLDER
    
for fold_idx in range(25,30):
    
    with h5py.File(path_to_split_data + str(fold_idx) + ".h5", 'r') as hf:
        if mode == "TRAIN":
            X = hf["X_train_transformed"][:,:num_components].astype(np.float64)
            labels = hf["y_train"][:]

        else:
            X = hf["X_valid_transformed"][:,:num_components].astype(np.float64)
            labels = hf["y_valid"][:]
            
        X_train =  hf["X_train_transformed"][:,:num_components].astype(np.float64)
        gene_symbols = hf["gene_symbols"][:]
        labels_names = hf["labels_names"][:]

        
    ####### PCA STUFF
    
    for transformation in ["KMeans", "PCA"]:
        
        num_components = 100

        print(path_to_new_files+ "%s/%i.h5"%(transformation, fold_idx))
        if not os.path.isdir(path_to_new_files + transformation + "/"):
            os.makedirs(path_to_new_files + transformation + "/")

        if transformation == "KMeans":
            kmeans = KMeans(n_clusters=num_components).fit(X_train.T)
            X_transformed = kmeans_centroids_for_test(X, kmeans.labels_).T
    
        elif transformation == "PCA":
            pca = PCA(n_components = num_components)
            pca.fit(X_train)
            X_transformed = pca.transform(X)[:, :num_components]


        print(labels.shape, X_transformed.shape)
        with h5py.File(path_to_new_files + "%s/%i.h5"%(transformation, fold_idx), 'w') as hf:
            hf.create_dataset("labels", data=labels)
            hf.create_dataset("outputs", data=X_transformed)
            hf.create_dataset("labels_names", data=labels_names)

last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/KMeans/25.h5
(1419, 11) (1419, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/PCA/25.h5
(1419, 11) (1419, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/KMeans/26.h5
(1402, 11) (1402, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/PCA/26.h5
(1402, 11) (1402, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/KMeans/27.h5
(1400, 11) (1400, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/PCA/27.h5
(1400, 11) (1400, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/KMeans/28.h5
(1413, 11) (1413, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/PCA/28.h5
(1413, 11) (1413, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/KMeans/29.h5
(1398, 11) (1398, 100)
last_shared_layer_transformations_TRAIN/origGE/unsupervised_methods/PCA/29.h

# Final model

In [14]:
SPECIFIC_FOLDER = "origGE"

SPECIFIC_FOLDER = "resid_01_GE_PmiRin"
SPECIFIC_FOLDER = "NEW_resid_then_01"

path_to_data = "../MTL_data/%s/ACT_MSBBRNA_ROSMAP_PCA.h5"%SPECIFIC_FOLDER
num_components=500
with h5py.File(path_to_data, 'r') as hf:
    X = hf["X"][:,:num_components].astype(np.float64)      
    Y = hf["Y"][:]
#     X = hf["ge_transformed"][:,:num_components].astype(np.float64)      
#     Y = hf["labels"][:]
    PCA_components = hf["PCA_components_"][:]
    gene_symbols = hf["gene_symbols"][:]
    labels_names = hf["labels_names"][:]
    


In [15]:
path_to_new_files = "last_shared_layer_transformations_final/%s/unsupervised_methods/"%SPECIFIC_FOLDER

for transformation in ["KMeans", "PCA"]:
    

    num_components = 100

    print(path_to_new_files+ "%s/final.h5"%(transformation))
    if not os.path.isdir(path_to_new_files + transformation + "/"):
        os.makedirs(path_to_new_files + transformation + "/")

    if transformation == "KMeans":
        kmeans = KMeans(n_clusters=num_components).fit(X.T)
        X_transformed = kmeans_centroids_for_test(X, kmeans.labels_).T

    elif transformation == "PCA":
        pca = PCA(n_components = num_components)
        pca.fit(X)
        X_transformed = pca.transform(X)[:, :num_components]


    print(Y.shape, X_transformed.shape)
    with h5py.File(path_to_new_files + "%s/final.h5"%(transformation), 'w') as hf:
        hf.create_dataset("labels", data=Y)
        hf.create_dataset("outputs", data=X_transformed)
        hf.create_dataset("labels_names", data=labels_names)

last_shared_layer_transformations_final/NEW_resid_then_01/unsupervised_methods/KMeans/final.h5
(1758, 11) (1758, 100)
last_shared_layer_transformations_final/NEW_resid_then_01/unsupervised_methods/PCA/final.h5
(1758, 11) (1758, 100)
